In [68]:
from time import sleep
import numpy as np
from PyTrilinos import Epetra

In [69]:
# initialize MPI
Comm  = Epetra.PyComm()

In [70]:
NumElements = 10
MapUnique = Epetra.Map(NumElements, 0, Comm)

In [71]:
print "Unique map"
print MapUnique

Unique map

Number of Global Elements  = 10
Number of Global Points    = 10
Maximum of all GIDs        = 9
Minimum of all GIDs        = 0
Index Base                 = 0
Constant Element Size      = 1

Number of Local Elements   = 10
Number of Local Points     = 10
Maximum of my GIDs         = 9
Minimum of my GIDs         = 0

         MyPID           Local Index        Global Index  
             0                 0                 0    
             0                 1                 1    
             0                 2                 2    
             0                 3                 3    
             0                 4                 4    
             0                 5                 5    
             0                 6                 6    
             0                 7                 7    
             0                 8                 8    
             0                 9                 9    



In [72]:
# Partially overlapping distribution
LocalElements = { 0: [0, 1, 2, 3, 4, 8],
                  1: [3, 4, 5, 6],
                  2: [6, 7, 8, 9] 
                }
MapOverl =  Epetra.Map(-1, LocalElements[Comm.MyPID()], 0, Comm)

In [73]:
print "Partially overlapping map"
print MapOverl

Partially overlapping map

Number of Global Elements  = 6
Number of Global Points    = 6
Maximum of all GIDs        = 8
Minimum of all GIDs        = 0
Index Base                 = 0
Constant Element Size      = 1

Number of Local Elements   = 6
Number of Local Points     = 6
Maximum of my GIDs         = 8
Minimum of my GIDs         = 0

         MyPID           Local Index        Global Index  
             0                 0                 0    
             0                 1                 1    
             0                 2                 2    
             0                 3                 3    
             0                 4                 4    
             0                 5                 8    



In [74]:
# Importer from overlapping to unique
Exporter = Epetra.Export(MapOverl, MapUnique)

In [75]:
# Create vectors
XOverl = Epetra.Vector(MapOverl)
XUnique = Epetra.Vector(MapUnique) #initialized to 0

In [76]:
XOverl[:] = Comm.MyPID() + 1
print "Max Value"
print Comm.MyPID(), XOverl.MaxValue()

In [77]:
Comm.Barrier()
def printVector(label, vector):
    Comm.Barrier()
    if Comm.MyPID() == 0: print label
    sleep(Comm.MyPID())
    print (Comm.MyPID(), vector)
printVector("Overlapping Vector", XOverl)

[ 1.  1.  1.  1.  1.  1.]
(0, 'Overlapping Vector')


In [78]:
XUnique.Export(XOverl, Exporter, Epetra.Average)

0

In [79]:
printVector("Unique Vector", XUnique)

[ 1.  1.  1.  1.  1.  0.  0.  0.  1.  0.]
(0, 'Unique Vector')


In [80]:
M=np.array([[ 2., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
           [-1.,  2., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
           [ 0., -1.,  2., -1.,  0.,  0.,  0.,  0.,  0.,  0.],
           [ 0.,  0., -1.,  2., -1.,  0.,  0.,  0.,  0.,  0.],
           [ 0.,  0.,  0., -1.,  2., -1.,  0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0., -1.,  2., -1.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.,  0., -1.,  2., -1.,  0.,  0.],
           [ 0.,  0.,  0.,  0.,  0.,  0., -1.,  2., -1.,  0.],
           [ 0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  2., -1.],
           [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  2.]])


In [81]:
MyGlobalElements  = MapUnique.MyGlobalElements()
Matrix            = Epetra.CrsMatrix(Epetra.Copy, MapUnique, 0)
for i in MyGlobalElements:
    if i > 0:
        Matrix[i, i - 1] = -1
    if i < NumElements - 1:
        Matrix[i, i + 1] = -1
    Matrix[i, i] = 2.
assert Matrix.FillComplete() == 0

In [82]:
for i in MyGlobalElements:
    print "%d: Matrix(%d, %d) = %e" %(Comm.MyPID(), i, i, Matrix[i, i])

0: Matrix(0, 0) = 2.000000e+00
0: Matrix(1, 1) = 2.000000e+00
0: Matrix(2, 2) = 2.000000e+00
0: Matrix(3, 3) = 2.000000e+00
0: Matrix(4, 4) = 2.000000e+00
0: Matrix(5, 5) = 2.000000e+00
0: Matrix(6, 6) = 2.000000e+00
0: Matrix(7, 7) = 2.000000e+00
0: Matrix(8, 8) = 2.000000e+00
0: Matrix(9, 9) = 2.000000e+00


In [83]:
XUnique2 = Epetra.Vector(MapUnique)
Matrix.Multiply(False, XUnique, XUnique2)

0

In [84]:
printVector("Unique Vector", XUnique2)

[ 1.  0.  0.  0.  1. -1.  0. -1.  2. -1.]
(0, 'Unique Vector')


In [85]:
from PyTrilinos import AztecOO
Solution = Epetra.Vector(MapUnique)
LinProb = Epetra.LinearProblem(Matrix, Solution, XUnique2)
IterSolver = AztecOO.AztecOO(LinProb)
IterSolver.Iterate(10, 1e-9)
printVector("Correct Solution", XUnique)
printVector("Iterative Solution", Solution)

[ 1.  1.  1.  1.  1.  0.  0.  0.  1.  0.]
(0, 'Solution')
